## Ordering features by importance (rxn-physicochemical properties)- GBC

In [32]:
import numpy as np
import pandas as pd
from src.config import raw_data_path, training_dataset_path, testing_dataset_path
import src.data.notebook_utils as utils


In [2]:
%cd ..

/home/belu/VS Workplace/2021-re-pendleton-perovskite


In [33]:
folder = 'notebooks/results/'
file = 'features.csv'
df = pd.read_csv(folder + file, index_col=1)

data = utils.read_data()
columns_by_type = utils.get_columns(data.columns)
predictors = columns_by_type['_feat_'] | columns_by_type['solUD'] 

In [34]:
# if feats_names_by_importance is index
df = df.reset_index().rename({'index':'feats_names_by_importance'}, axis='columns')

In [35]:
df.head()

,feats_names_by_importance,feats_importance_values
0,['_rxn_M_organic' '_rxn_M_inorganic' '_rxn_M_a...,[0.22842081 0.18604353 0.17251794 0.1607025 0...
1,['_rxn_M_organic' '_rxn_M_inorganic' '_rxn_M_a...,[2.26817202e-01 2.09639989e-01 1.63145387e-01 ...
2,['_rxn_M_organic' '_rxn_M_inorganic' '_rxn_M_a...,[2.23563984e-01 1.93830753e-01 1.62923154e-01 ...
3,['_rxn_M_organic' '_rxn_M_inorganic' '_feat_AS...,[2.28278260e-01 2.20237248e-01 1.55120340e-01 ...
4,['_rxn_M_organic' '_rxn_M_inorganic' '_rxn_M_a...,[2.22716525e-01 2.01786642e-01 1.62002408e-01 ...


In [36]:
len(predictors)

75

In [60]:
def row_to_list(x):
    l = x[1:-1].split(' ')
    return [f.rstrip()[1:-1] for f in l]

In [61]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def convert_to_float(f):
    if isfloat(f): return float(f)
    return 0

In [62]:
def feat_values_to_list(x):
    l = x[1:-1].split(' ')
    return([convert_to_float(f.rstrip()) for f in l])

In [63]:
weight_per_feature = {}
for index, row in df.iterrows():
    feat_names = row_to_list(row['feats_names_by_importance'])
    feat_values = feat_values_to_list(row['feats_importance_values'])
    for feat_name in feat_names: 
        if feat_name not in weight_per_feature.keys(): weight_per_feature[feat_name] = 0
    i = 0
    while(float(feat_values[i])>0.000001 and i < min(len(feat_names),len(feat_values)) ):        
        weight_per_feature[feat_names[i]] += float(feat_values[i])
        i+=1
    

In [66]:
df_weight_per_feature = pd.DataFrame(list(weight_per_feature.items()),columns = ['feature','weight'])

In [67]:
df_weight_per_feature['weight'] = df_weight_per_feature['weight'].apply(lambda x: x/len(df))

In [68]:
df_weight_per_feature.head()

,feature,weight
0,_rxn_M_organic,0.222841
1,_rxn_M_inorganic,0.202689
2,_rxn_M_acid,0.162786
3,_feat_ASA-,0.146227
4,_feat_BalabanIndex,0.026015


In [69]:
df_weight_per_feature.to_csv(folder + 'fetures_sorted_by_weight.csv', index=None)